In [8]:
#This program will take a df and strip it of columns so it is ready for the lift over from hg18 ro hg 19 
#Tool is located http://genome.ucsc.edu/cgi-bin/hgLiftOver

#import libraries 
import pandas as pd 
import numpy as np

#Declare file locations
root = "/Users/npatel/Desktop/Dr.Chen_Lab/Neel_Data_Results/Task1-Eagle_Data"

#import dataframe 
hg18_df = pd.read_csv("/Users/npatel/Desktop/Dr.Chen_Lab/Neel_Data_Results/Task1-Eagle_Data/EAGLE_aggression_Public_2_FINAL_MERGED(without Duplicates).txt", sep = '\s+' )
missing = pd.read_csv("/Users/npatel/Desktop/Dr.Chen_Lab/Data/EAGLE_aggression_Public_2_FOUND.csv")
chromData = pd.read_csv("/Users/npatel/Desktop/Dr.Chen_Lab/Genome_Data/ALL-snps_hg18.txt", sep='\s+')
liftover = pd.read_csv("/Users/npatel/Desktop/Dr.Chen_Lab/Neel_Data_Results/Task1-Eagle_Data/hglft_genome_3611b_e19c90.bed",sep='\s+',names = ['chrom', 'chromStart' ,'chromEnd'])
chromData2 = pd.read_csv("/Users/npatel/Desktop/Dr.Chen_Lab/Genome_Data/snp126.txt", sep='\s+', names = ['chrom', 'chromStart' ,'chromEnd','SNP'], usecols = [1,2,3,4])
failedConv = pd.read_csv("/Users/npatel/Desktop/Dr.Chen_Lab/Neel_Data_Results/Task1-Eagle_Data/hglft_genome_failed_conversions.txt", sep='\s+', names = ['chrom', 'chromStart' ,'chromEnd'])
hg18_Cov = pd.read_csv("/Users/npatel/Desktop/Dr.Chen_Lab/Neel_Data_Results/Task1-Eagle_Data/hglft_genome_272ca_75ff30.bed", sep = '\s+', names = ['chrom', 'chromStart' ,'chromEnd'])
hg18_Cov_again = pd.read_csv("/Users/npatel/Desktop/Dr.Chen_Lab/Neel_Data_Results/Task1-Eagle_Data/hglft_genome_2b775_88b3f0.bed", sep = '\s+', names = ['chrom', 'chromStart' ,'chromEnd'])


/Users/npatel/Desktop/Dr.Chen_Lab/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
#Set hg18 data to BED format for liftover tool 
hg18_BED = 'chr' + hg18_df['CHR'].astype(str) + ' ' + hg18_df['BP'].astype(str)

#Merge hg18 data and chromData2 on SNP to include the SNP start info.
df = pd.merge(hg18_df, chromData2, on = "SNP",  how='outer', indicator=True)
final = df[['chrom', 'chromStart', 'chromEnd', 'SNP']]

#Set up new data frame with a single string to be BED formate for liftover tool
exportList = final['chrom'].astype(str) + ' ' + final['chromStart'].astype(str) + ' ' + final['chromEnd'].astype(str)
final

#Export to txt for liftover
exportList.to_csv(root + '/' + "exportLiftList(hg18).txt", index = False,)
final

,chrom,chromStart,chromEnd,SNP
0,chr1,5875931,5875932,rs11122104
1,chr1,94309882,94309883,rs12758774
2,chr1,207960665,207960666,rs6672256
3,chr1,233039174,233039175,rs822841
4,chr1,212439579,212439580,rs1223766
...,...,...,...,...
12351936,chrY,57442439,57442440,rs28462370
12351937,chrY,57442478,57442479,rs28607775
12351938,chrY,57442486,57442487,rs28476519
12351939,chrY,57442497,57442498,rs28628009



#final['chromEnd'] = final['chromEnd'].astype(object)
failedConv['chromEnd'] = failedConv['chromEnd'].astype(int)
final['chromEnd'] = final['chromEnd'].astype(int)

cond = final['chromEnd'].isin(failedConv['chromEnd'])
final.drop(final[cond].index, inplace = True)

exportList = final['chrom'].astype(str) + ' ' + final['chromStart'].astype(str) + ' ' + final['chromEnd'].astype(str)
exportList.to_csv(root + '/' + "exportLiftList(hg18)_AfterDeletingFailures.txt", index = False,)
final
